To copy this template: File -> Save a Copy in Drive

***DISCLAIMER**: In case of any discrepancy in the assignment instruction, please refer to the `PDF` document.*

# Problem 2 - Neural Network Training and Backpropagation

## 2.1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline
#Load data and create X and y variables
data = loadmat('/content/sample_data/ex3data1.mat')
X = data['X']
y = data['y']
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [ ]:
X = data['X']
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

In [ ]:
# transorm y data - vector of length k where index n is "hot" (1) while the rest are zero
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(5000, 10)

In [ ]:
# needs to be more sensitive - account for factor of 2
def sigmoid(z):
    return 1 / (1 + np.exp(-2*z))

## 2.2

In [ ]:
# propogate with two hidden layers - additional a and z values
def forward_propagate(X, theta1, theta2, theta3):
    m = X.shape[0]

    # Input layer to Hidden Layer 1
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 @ theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)

    # Hidden Layer 1 to Hidden Layer 2
    z3 = a2 @ theta2.T
    a3 = np.insert(sigmoid(z3), 0, values=np.ones(m), axis=1)

    # Hidden Layer 2 to Output Layer
    z4 = a3 @ theta3.T
    h = sigmoid(z4)

    return a1, z2, a2, z3, a3, z4, h

## 2.3

In [ ]:
def cost_with_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1_start = 0
    theta1_end = hidden_size1 * (input_size + 1)
    theta2_end = theta1_end + (hidden_size1 + 1) * hidden_size2
    theta1 = np.matrix(np.reshape(params[theta1_start:theta1_end], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[theta1_end:theta2_end], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[theta2_end:], (num_labels, (hidden_size2 + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:, 1:], 2)) +
                                             np.sum(np.power(theta2[:, 1:], 2)) +
                                             np.sum(np.power(theta3[:, 1:], 2)))

    return J



In [ ]:
def cost_without_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    end_of_theta1 = hidden_size1 * (input_size + 1)
    end_of_theta2 = end_of_theta1 + hidden_size2 * (hidden_size1 + 1)

    theta1 = np.matrix(np.reshape(params[:end_of_theta1], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_of_theta1:end_of_theta2], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[end_of_theta2:], (num_labels, (hidden_size2 + 1))))

    # Run the forward propagation
    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    # Compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    return J



In [ ]:
# initial setup
input_size = 400
hidden_size1 = 20  # Size of the first hidden layer
hidden_size2 = 20  # Size of the second hidden layer
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params_size = (input_size + 1) * hidden_size1 + (hidden_size1 + 1) * hidden_size2 + (hidden_size2 + 1) * num_labels
params = (np.random.random(size=params_size) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
end_hidden1 = hidden_size1 * (input_size + 1)
end_hidden2 = end_hidden1 + hidden_size2 * (hidden_size1 + 1)
theta1 = np.matrix(np.reshape(params[:end_hidden1], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[end_hidden1:end_hidden2], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(params[end_hidden2:], (num_labels, (hidden_size2 + 1))))

print(theta1.shape, theta2.shape, theta3.shape)

(20, 401) (20, 21) (10, 21)


In [ ]:
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
a1.shape, z2.shape, a2.shape, z3.shape, a3.shape, z4.shape, h.shape

((5000, 401),
 (5000, 20),
 (5000, 21),
 (5000, 20),
 (5000, 21),
 (5000, 10),
 (5000, 10))

In [ ]:
cost_without_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot)

6.767207292541944

In [ ]:
cost_with_regularization(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)

6.771734275017079

## 2.4

In [ ]:
def sigmoid_gradient(z):
    return 2 * np.multiply(sigmoid(z), (1 - sigmoid(z)))

## 2.5

In [ ]:
def backprop_unregularized(params, input_size, hidden_size1, hidden_size2, num_labels, X, y):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    end_of_theta1 = hidden_size1 * (input_size + 1)
    end_of_theta2 = end_of_theta1 + hidden_size2 * (hidden_size1 + 1)

    theta1 = np.matrix(np.reshape(params[:end_of_theta1], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_of_theta1:end_of_theta2], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[end_of_theta2:], (num_labels, (hidden_size2 + 1))))

    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(np.insert(z3t, 0, values=np.ones(1))))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(np.insert(z2t, 0, values=np.ones(1))))

        delta1 += (d2t[:,1:]).T * a1t
        delta2 += (d3t[:,1:]).T * a2t
        delta3 += d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad

In [ ]:
def backprop_regularized(params, input_size, hidden_size1, hidden_size2, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    end_of_theta1 = hidden_size1 * (input_size + 1)
    end_of_theta2 = end_of_theta1 + hidden_size2 * (hidden_size1 + 1)

    theta1 = np.matrix(np.reshape(params[:end_of_theta1], (hidden_size1, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[end_of_theta1:end_of_theta2], (hidden_size2, (hidden_size1 + 1))))
    theta3 = np.matrix(np.reshape(params[end_of_theta2:], (num_labels, (hidden_size2 + 1))))

    a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)

    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    delta3 = np.zeros(theta3.shape)

    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) +
                                             np.sum(np.power(theta2[:,1:], 2)) +
                                             np.sum(np.power(theta3[:,1:], 2)))

    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        z3t = z3[t,:]
        a3t = a3[t,:]
        ht = h[t,:]
        yt = y[t,:]

        d4t = ht - yt
        d3t = np.multiply((theta3.T * d4t.T).T, sigmoid_gradient(np.insert(z3t, 0, values=np.ones(1))))
        d2t = np.multiply((theta2.T * d3t[:,1:].T).T, sigmoid_gradient(np.insert(z2t, 0, values=np.ones(1))))

        delta1 += (d2t[:,1:]).T * a1t
        delta2 += (d3t[:,1:]).T * a2t
        delta3 += d4t.T * a3t

    delta1 = delta1 / m
    delta2 = delta2 / m
    delta3 = delta3 / m

    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    delta3[:,1:] = delta3[:,1:] + (theta3[:,1:] * learning_rate) / m

    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2), np.ravel(delta3)))

    return J, grad


In [ ]:
J, grad = backprop_unregularized(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot)
J, grad.shape

(6.767207292541944, (8650,))

In [ ]:
J, grad = backprop_regularized(params, input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot, learning_rate)
J, grad.shape

(6.771734275017079, (8650,))

## 2.6

In [ ]:
from scipy.optimize import minimize
import pandas as pd

#input feature
input_size = 400
#first hidden layer units
hidden_size1 = 20
#second hidden layer units
hidden_size2 = 20
# classes
num_labels = 10
learning_rate = 1

# Randomly initialize weights
params = (np.random.random(size=hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1) + num_labels * (hidden_size2 + 1)) - 0.5) * 0.25

# Minimize the objective function
fmin = minimize(fun=backprop_unregularized, x0=params, args=(input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot),
                method='TNC', jac=True, options={'maxiter': 250})

# Extract trained parameters
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size1 * (input_size + 1)], (hidden_size1, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size1 * (input_size + 1):hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1)], (hidden_size2, (hidden_size1 + 1))))
theta3 = np.matrix(np.reshape(fmin.x[hidden_size1 * (input_size + 1) + hidden_size2 * (hidden_size1 + 1):], (num_labels, (hidden_size2 + 1))))

<ipython-input-50-d927eae6725c>:18: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun=backprop_unregularized, x0=params, args=(input_size, hidden_size1, hidden_size2, num_labels, X, y_onehot),
<ipython-input-46-9e4380073be5>:22: RuntimeWarning: divide by zero encountered in log
  second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
<ipython-input-46-9e4380073be5>:22: RuntimeWarning: invalid value encountered in multiply
  second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))


## 2.7

In [ ]:
a1, z2, a2, z3, a3, z4, h = forward_propagate(X, theta1, theta2, theta3)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

In [ ]:
accuracy_value = float(sum(a == b for a, b in zip(y_pred, y))) / len(y)
print(f'accuracy = {accuracy_value * 100:.2f}%')


accuracy = 100.00%


## 2.8

**Answer:**'
The 3 layer NN I created achieved an 100.00% accuracy as compared to the example 2 layer NN which had a 99.22% accuracy. This indicates an improvement in capturing more features and patterns in the data with an additional layer of hidden network. However, the additiional layer did improve 100% accuracy may indicate there is overfitting as this is very high. Meaning it may not be worth it to increase the number of hidden layers.
